In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin, urlunparse # urlparseなどを追加
import time
from collections import deque # dequeを追加
import re # 正規表現モジュールを追加

# --- 定数と初期設定 ---
# 武蔵野大学のトップページのURL
START_URL = "https://www.musashino-u.ac.jp/"
# アクセスするWebサイトのベースドメイン
BASE_DOMAIN = urlparse(START_URL).netloc

# ページアクセス間の待機時間（秒）- ユーザーの指示により0.5秒
SLEEP_TIME = 0.5 

# 除外するファイル拡張子リスト（小文字で統一）
EXCLUDE_EXTENSIONS = (
    '.jpg', '.jpeg', '.png', '.gif', '.bmp', '.svg',  # 画像
    '.pdf', '.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx', # ドキュメント
    '.zip', '.rar', '.tar', '.gz', # 圧縮ファイル
    '.mp3', '.mp4', '.avi', '.mov', # メディアファイル
    '.css', '.js', # スタイルシート、JavaScriptファイル (通常はクロール対象外)
    '.ico', '.xml', '.txt' # その他のファイル（robots.txtなど）
)

# --- データ構造の初期化 ---
# 探索済みURLを格納するセット（重複アクセス防止、正規化されたURLを格納）
visited_urls = set()
# 探索待ちURLを格納するキュー（BFS: 幅優先探索のため）
url_queue = deque()
# 結果を格納する辞書 {URL: TITLE}
sitemap = {} 

# --- URLの正規化ヘルパー関数 ---
def normalize_url(base_url, relative_url):
    """
    相対URLを絶対URLに変換し、クエリパラメータとフラグメントを除去して正規化します。
    """
    try:
        # 相対URLを絶対URLに変換
        absolute_url = urljoin(base_url, relative_url)
        
        # URLをパース
        parsed_url = urlparse(absolute_url)
        
        # クエリパラメータとフラグメントを除去
        # scheme, netloc, path, params, query, fragment
        normalized = parsed_url._replace(query="", fragment="")
        
        # URLを再構築
        return urlunparse(normalized)
    except Exception as e:
        # デバッグ時にコメント解除すると詳細なエラーが見れます
        # print(f"URL正規化中にエラーが発生しました: {relative_url} -> {e}") 
        return None

# --- メインのクロール関数 ---
def crawl_website():
    """
    指定されたスタートURLから同一ドメイン内のリンクを幅優先探索で辿り、
    URLとタイトルを収集する関数。
    """
    
    # スタートURLを正規化してキューに追加
    start_normalized_url = normalize_url("", START_URL) # ベースURLは空でOK
    if start_normalized_url:
        url_queue.append(start_normalized_url)
        visited_urls.add(start_normalized_url) # スタートURLも訪問済みとしてマーク

    # 探索対象がなくなるまでループ
    while url_queue:
        # キューから次のURLを取り出す
        current_url = url_queue.popleft() 
        
        # 負荷軽減のための待機
        print(f"探索中: {current_url}")
        time.sleep(SLEEP_TIME) 

        try:
            # ウェブサイトにアクセス
            response = requests.get(current_url, timeout=10) # 10秒でタイムアウト
            response.raise_for_status() # HTTPエラー（4xx, 5xx）があれば例外を発生させる

            # エンコーディングを正しく設定（文字化けを防ぐ）
            # レスポンスヘッダからエンコーディングを判断し、不明な場合は推測
            response.encoding = response.apparent_encoding if response.encoding is None else response.encoding
            
            # BeautifulSoupでHTMLを解析
            soup = BeautifulSoup(response.text, 'html.parser')

            # ページのタイトルを取得し、サイトマップに格納
            page_title_tag = soup.find('title')
            page_title = page_title_tag.get_text(strip=True) if page_title_tag else "タイトルなし"
            sitemap[current_url] = page_title # ここでタイトルを格納

            # ページ内の全てのリンクを取得
            # find_all('a', href=True) で、href属性を持つ<a>タグ（リンク）を全て見つける
            for link in soup.find_all('a', href=True):
                href = link['href']
                
                # リンクを正規化（絶対URLに変換し、クエリやフラグメントを除去）
                absolute_normalized_url = normalize_url(current_url, href)
                
                if not absolute_normalized_url:
                    continue # 正規化に失敗した場合はスキップ

                # リンク先のドメインをチェック
                parsed_link_domain = urlparse(absolute_normalized_url).netloc
                
                # 同一ドメイン (BASE_DOMAIN と一致またはサブドメイン) かつ、まだ訪問していないURL、
                # かつ、除外ファイル拡張子でないもののみを対象とする
                # (parsed_link_domain == BASE_DOMAIN) は完全一致
                # (parsed_link_domain.endswith(f".{BASE_DOMAIN}")) はサブドメイン（例: sub.musashino-u.ac.jp）をカバー
                is_same_domain = (parsed_link_domain == BASE_DOMAIN) or \
                                 (parsed_link_domain.endswith(f".{BASE_DOMAIN}") and len(parsed_link_domain) > len(BASE_DOMAIN))
                
                # ファイル拡張子チェック
                is_excluded_file = any(absolute_normalized_url.lower().endswith(ext) for ext in EXCLUDE_EXTENSIONS)

                if is_same_domain and \
                   absolute_normalized_url not in visited_urls and \
                   not is_excluded_file:
                    
                    print(f"  リンク検出（キューに追加）: {absolute_normalized_url}")
                    url_queue.append(absolute_normalized_url)
                    visited_urls.add(absolute_normalized_url) # キューに追加した時点で訪問済みとしてマーク

        except requests.exceptions.HTTPError as e:
            print(f"エラー: HTTPステータス {e.response.status_code} - {e} - URL: {current_url}")
        except requests.exceptions.ConnectionError as e:
            print(f"エラー: 接続に失敗しました - {e} - URL: {current_url}")
        except requests.exceptions.Timeout as e:
            print(f"エラー: リクエストがタイムアウトしました - {e} - URL: {current_url}")
        except requests.exceptions.RequestException as e:
            print(f"エラー: その他のリクエスト関連エラー - {e} - URL: {current_url}")
        except Exception as e:
            print(f"エラー: 予期せぬエラーが発生しました - {e} - URL: {current_url}")

# --- 実行開始 ---
print("--- サイトマップ抽出開始 ---")
print(f"ターゲットURL: {START_URL}")
print(f"ベースドメイン: {BASE_DOMAIN}")
print(f"ページアクセス間隔: {SLEEP_TIME}秒")

crawl_website() # 関数呼び出しで探索を開始

print("\n--- サイトマップ抽出完了 ---")
print(f"抽出されたページ数: {len(sitemap)}")

# 辞書型変数を print() で表示
print("\n=== 抽出されたサイトマップ ===")
for url, title in sitemap.items():
    print(f"URL: {url}\nTitle: {title}\n---")

--- サイトマップ抽出開始 ---
ターゲットURL: https://www.musashino-u.ac.jp/
ベースドメイン: www.musashino-u.ac.jp
ページアクセス間隔: 0.5秒
探索中: https://www.musashino-u.ac.jp/
  リンク検出（キューに追加）: https://www.musashino-u.ac.jp/access.html
  リンク検出（キューに追加）: https://www.musashino-u.ac.jp/admission/request.html
  リンク検出（キューに追加）: https://www.musashino-u.ac.jp/contact.html
  リンク検出（キューに追加）: https://www.musashino-u.ac.jp/prospective-students.html
  リンク検出（キューに追加）: https://www.musashino-u.ac.jp/students.html
  リンク検出（キューに追加）: https://www.musashino-u.ac.jp/alumni.html
  リンク検出（キューに追加）: https://www.musashino-u.ac.jp/parents.html
  リンク検出（キューに追加）: https://www.musashino-u.ac.jp/business.html
  リンク検出（キューに追加）: https://www.musashino-u.ac.jp/guide/
  リンク検出（キューに追加）: https://www.musashino-u.ac.jp/guide/profile/
  リンク検出（キューに追加）: https://www.musashino-u.ac.jp/guide/activities/
  リンク検出（キューに追加）: https://www.musashino-u.ac.jp/guide/campus/
  リンク検出（キューに追加）: https://www.musashino-u.ac.jp/guide/facility/
  リンク検出（キューに追加）: https://www.musashino-u.ac.jp/gu